<a href="https://colab.research.google.com/github/salacamifda/Actividad-de-pipelines/blob/main/Actividad_de_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Imports
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
set_config(display='diagram')


In [11]:
path = '/content/Cereal with missing values.xlsx'
df = pd.read_excel(path)
df.head()


,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2.0,2.0,180.0,1.5,10.5,10.0,70.0,25.0,1.0,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3.0,2.0,NaN,2.0,18.0,NaN,100.0,25.0,3.0,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6.0,2.0,290.0,2.0,17.0,1.0,105.0,25.0,1.0,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1.0,3.0,210.0,0.0,13.0,9.0,45.0,25.0,2.0,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3.0,2.0,140.0,2.0,13.0,7.0,105.0,25.0,3.0,1.00,0.50,40.400208


In [12]:
#inspect the data
print(df.info(), '\n')
print(df.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     float64
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

# Actividad de pipelines

¿Qué tan bien se pueden predecir las calorías basándose en el fabricante, tipo de cereal, gramos de grasa, gramos de azúcares y el peso en onzas por una porción del cereal? 

Hasta ahora, solo completarán los pasos de preprocesamiento para esta tarea.

Deberán:

* Definir la característica (X) y objetivo (y).


In [13]:
# dividan la característica y el objetivo y realicen un train/test split.
X = df.drop(columns=['calories per serving'])
y = df['calories per serving']

* Utilizar train test split en los datos para prepararlos para aprendizaje automático.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

* Identificar cada característica como numérica, ordinal o nominal. (Entreguen esta pregunta en una celda de texto en un cuaderno de Colab).


In [8]:
df['Manufacturer'].value_counts()

Kelloggs                       23
General Mills                  22
Post                            9
Quaker Oats                     8
Ralston Purina                  8
Nabisco                         6
American Home Food Products     1
Name: Manufacturer, dtype: int64

In [9]:
df['type'].value_counts()

Cold    65
Hot      3
Name: type, dtype: int64

-Características numéricas:
 3   calories per serving                             
 4   grams of protein                                 
 5   grams of fat                                     
 6   milligrams of sodium                             
 7   grams of dietary fiber                           
 8   grams of complex carbohydrates                  
 9   grams of sugars                                  
 10  milligrams of potassium                          
 11  vitamins and minerals (% of FDA recommendation)  
 12  Display shelf                                    
 13  Weight in ounces per one serving                 
 14  Number of cups in one serving                    
 15  Rating of cereal                

-Características ordinal:
 NA

-Características nominal:
 1   name
 2   Manufacturer
 3   Type

* Utilizar pipelines y transformadores de columnas para completar las siguientes tareas:

-Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas y la estrategia “most_frequent” para las columnas categóricas.

-Realizar una codificación one-hot a las características nominales.

-Escalar las columnas numéricas.


In [16]:
#Realizar selectores de columnas

cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [19]:
# Imputers
most_frequent_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)



Instanciar el pipeline

In [20]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [21]:
# Categorical pipeline
categorical_pipe = make_pipeline(most_frequent_imputer, ohe)
categorical_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [24]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor



ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f83bd6295d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f83bd6296d0>)])

* Todos los datos de preprocesamiento deben estar contenidos en un único objeto de preprocesamiento.

In [25]:
# Transformador de datos
# fit on train
preprocessor.fit(X_train)



ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f83bd6295d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f83bd6296d0>)])

In [26]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)



*Utilizar el objeto de preprocesamiento para transformar los datos de manera adecuada, evitar la fuga de datos para dejarlos listos para el modelado.

*Muestren los arrays de NumPy resultantes.

In [27]:
#Inspeccionar el resultado
# Comprueben los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')
X_train_processed



0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (57, 77)




array([[-1.30301442, -0.97467943,  0.56162348, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.68120871, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378, -0.97467943,  1.99664622, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.25808288,  1.94935887, -0.03630266, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.97467943, -0.15588789, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.08328257, ...,  0.        ,
         1.        ,  0.        ]])